In [2]:
"""
Created on Tue Oct 27 20:19:13 2020

@author: Brandon McNabb (bmcnabb@eoas.ubc.ca)
"""

'\nCreated on Tue Oct 27 20:19:13 2020\n\n@author: Brandon McNabb (bmcnabb@eoas.ubc.ca)\n'

## A script for post-processing data used in "NESAP_Sulfur_Climatology_v5.ipynb". Data files were obtained from the following sources:
---
### PMEL (DMS 1997-2017)
 - https://saga.pmel.noaa.gov/dms/

---
### Aqua MODIS (2002-2017) & SeaWiFS/Aqua TERRA (1997-2002)
 
#### SSHA (Sea Surface Height Anomalies)
 - https://podaac-opendap.jpl.nasa.gov/opendap/allData/merged_alt/L4/cdr_grid/ssh_grids_v1812_2019010412.nc.html
 - https://podaac.jpl.nasa.gov/dataset/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL1812

#### Chlorophyll-a, Calcite (PIC), photosyntheically active radiation (daily=PAR, instantaneous=iPAR), sea surface temperature (SST), fluorescence line height (nFLH), diffuse attenuation coefficients (kd)
 - https://oceancolor.gsfc.nasa.gov/l3/
---
### Copernicus wind speeds (2007-2017)
 - https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=WIND_GLO_PHY_CLIMATE_L4_REP_012_003
---
### NPP (VGPM model)
##### From Aqua MODIS (2002-2017):
 - http://orca.science.oregonstate.edu/1080.by.2160.monthly.hdf.vgpm.m.chl.m.sst.php

##### From SeaWiFS (1997-2002):
 - http://orca.science.oregonstate.edu/1080.by.2160.monthly.hdf.vgpm.s.chl.a.sst.php
---
### World Ocean Atlas (WOA18) (sea surface nitrate SSN)
 - https://www.ncei.noaa.gov/access/world-ocean-atlas-2018/
---
### MIMOC Climatology (sea surface salinity (SSS), mixed layer depth (MLD)):
 - https://www.pmel.noaa.gov/mimoc/
---
### ETOPO2 Bathymetry
 - https://rda.ucar.edu/datasets/ds759.3/
---
### NCP Algorithm:
Li, Z., & Cassar, N. (2016). Satellite estimates of net community production based on O 2 /Ar observations and comparison to other estimates. Global Biogeochemical Cycles, 30(5), 735–752. https://doi.org/10/f8v6bh

---
### NPQ-corrected Fluorescence Yield Algorithm:
Behrenfeld, M. J., Westberry, T. K., Boss, E. S., O’Malley, R. T., Siegel, D. A., Wiggert, J. D., Franz, B. A., McClain, C. R., Feldman, G. C., Doney, S. C., Moore, J. K., Dall’Olmo, G., Milligan, A. J., Lima, I., & Mahowald, N. (2009). Satellite-detected ﬂuorescence reveals global physiology of ocean phytoplankton. Biogeosciences, 6, 16. https://doi.org/10/fdn3f2

---

## Import Packages

In [ ]:
import timeit
runtime_start = timeit.default_timer() # start the clock

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import scipy.interpolate
import os
from dateutil import parser
import xarray as xr
import dask.array as da
import dask.dataframe as dd
from pyhdf.SD import SD, SDC
import datetime

## Define constants and parameters

In [ ]:

#~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
# *** Make sure lat/lons bounds are the same as analysis script!! ***
#~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

#### Spatial grid resolution (degrees):
grid = 0.25

#### Define lat/lon constraints

# Line P/La Perouse area:
# Set directory to write data files to:
# write_dir = 'C:/Users/bcamc/OneDrive/Desktop/Python/Projects/py_eosc510/Final Project/NEPac_Data/'
# min_lat = 43
# max_lat = 60
# min_lon = -147
# max_lon = -122

# Full NESAP region:
# Set directory to write data files to:
# write_dir = 'C:/Users/bcamc/OneDrive/Desktop/Python/Projects/py_eosc510/Final Project/NESAP_Data/'
# write_dir = 'C:/Users/bcamc/OneDrive/Desktop/Python/Projects/py_eosc510/Final Project/NESAP_Data_v2/'
write_dir = 'C:/Users/bcamc/OneDrive/Desktop/Python/Projects/py_eosc510/Final Project/NESAP_Data_res/'

min_lat = 40
max_lat = 61
min_lon = -180
max_lon = -122

#### Define bins
latbins = np.arange(min_lat,max_lat+grid,grid)
lonbins = np.arange(min_lon,max_lon+grid,grid)

## Extract dimensions of data

In [ ]:

start = timeit.default_timer() # start the clock
#-----------------------------------------------------------------------------
# Set path:
# file directory shape:
# -/MODIS
#       |-/Data
#           |-/1_Chl
#               |-/*.nc
#           |-/2_PIC
#               |-/*.nc
#           |-/3_SST
#               |-/*.nc
#           |-/4_PAR
#               |-/*.nc
#           |-/5_Kd
#               |-/*.nc
#           |-/6_POC
#               |-/*.nc
#           |-/7_FLH
#               |-/*.nc
#           |-/8_SSHA
#               |-/*.nc
#           |-/9_NPP
#               |-/*.hdf
#           |-/10_MIMOC_MLD
#               |-/*.hdf
#           |-/11_MIMOC_TS
#               |-/*.hdf

directory = 'E:/Satellite/Data_Subset_NESAP' # E: or C:, depending if using hard drive/thumb drive
folders = os.listdir(directory) # calls specific datasets grouped in folders
num_folders = 14 # set this to the number of variables
#-----------------------------------------------------------------------------

print('finding dimensions...')

# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
# preallocate variable:
dims = np.empty((num_folders,4))
# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
for count, folder in enumerate(folders):
    path = os.path.join(directory, folder)
    for subdir, dirs, files in os.walk(path):
        for n, file in enumerate(files):
            if count < 9 and n == 1:
                # if count <= 4:
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # Pull the lat/lon from it:
                lat = check.coords['lat'].values
                lon = check.coords['lon'].values
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
            if count == 9 and n == 1: # Data configured differently for SSHA data
                # if count <= 4:
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # Pull the lat/lon from it:
                lat = check.coords['Latitude'].values
                lon = check.coords['Longitude'].values
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
            if count == 10 and n == 1: # Data configured differently for SSH data
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath_SSH = subdir + os.sep + file
                check = xr.open_dataset(filepath_SSH)
                # lets pull the lat/lon from it:
                lat = check.coords['latitude'].values
                lon = check.coords['longitude'].values-180 # convert to W/E coords
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
            if count == 11 and n == 1: # Data configured differently for NPP data
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath_NPP = subdir + os.sep + file
                vars_ = SD(filepath_NPP, SDC.READ)
                # data = np.flipud(vars_.select('npp').get())
                # lets pull the lat/lon from it:
                n = 180/1080
                lat = np.arange(-90,90,n)
                lon = np.arange(-180,180,n)
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
            if count == 12 or count == 11 and n == 1: # Data configured differently for MLD data
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # lets pull the lat/lon from it:
                lat = check.LATITUDE.values
                lon = check.LONGITUDE.values-180 # convert to W/E coords
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
            else:
                pass
    # extract number of files (datasets) per folder + lat/lon dimensions:
    dims[count] = np.array([int(count), int(np.size(files)), int(data_proc.shape[0]*np.size(files)), int(data_proc.shape[1])])
    print('\nData length:',str(data_proc.shape[0]))
    print('In folder ' + '"' + folder + '":')
    print('Number of datasets = '+ str(np.size(files)))
end = timeit.default_timer() # stop the clock
print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~')
print('Execution time:')
print(str(round(end-start,5)),'secs')
print(str(round((end-start)/60,5)),'mins')
print(str(round((end-start)/3600,5)),'hrs')
del check

## Pre-allocate variables

In [ ]:
# Disk space can't handle computing all the files at once (>30 gb worth),
# so use Dask package to parallel compute over CPUs
#-----------------------------------------------------------------------------
# Pre-allocate extraction variables as a dict using Dask:
variables = {'chl':dd.from_array(da.empty((dims[0,2],dims[0,3]))),
             'PIC':dd.from_array(da.empty((dims[1,2],dims[1,3]))),
             'SST':dd.from_array(da.empty((dims[2,2],dims[2,3]))),
             'PAR':dd.from_array(da.empty((dims[3,2],dims[3,3]))),
             'Kd':dd.from_array(da.empty((dims[4,2],dims[4,3]))),
             'POC':dd.from_array(da.empty((dims[5,2],dims[5,3]))),
             'FLH':dd.from_array(da.empty((dims[6,2],dims[6,3]))),
             'iPAR':dd.from_array(da.empty((dims[7,2],dims[7,3]))),
             'CDOM':dd.from_array(da.empty((dims[8,2],dims[8,3]))),
             'SSHA':dd.from_array(da.empty((dims[9,2],dims[9,3]))),
             'SSH':dd.from_array(da.empty((dims[10,2],dims[10,3]))),
             'NPP':dd.from_array(da.empty((dims[11,2],dims[11,3]))),
             'MLD':dd.from_array(da.empty((dims[12,2],dims[12,3]))),
             'SAL':dd.from_array(da.empty((dims[13,2],dims[13,3])))}

## Extract MODIS/SeaWiFS/MIMOC from file directory

In [ ]:
#-----------------------------------------------------------------------------
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Beginning extraction loop...')
start = timeit.default_timer() # start the clock
for count, (k, folder) in enumerate(zip(variables, folders)):
    print('\nAccessing ' + '"' + folder + '"' + ' folder...')
    print()
    path = os.path.join(directory, folder)
    if count == 0 or count==10: # My PC is RAM limited so this line bypasses looping at higher res (0.25x0.25) below to extract one variable at a time, saving memory space; if computing resources are not limited, comment out to extract all variables at once
        for subdir, dirs, files in os.walk(path):
            for n, file in enumerate(files):
                filepath = subdir + os.sep + file
                # Print status: starting iteration
                print(folder + ' (' + str(count+1) + ' of ' + str(np.size(folders)) + ' folders)' + ': Extracting... ' + str(n+1) +' of ' + str(np.size(files)))
                #-----------------------------------------------------------------
                # Extract data
                if count < 12:
                    if count <= 9:
                        # Open netCDF file
                        vars_ = xr.open_dataset(filepath)
                        if count == 0:
                            # Chl:
                            data = vars_.chlor_a.values
                        if count == 1:
                            # PIC:
                            data = vars_.pic.values
                        if count == 2:
                            # SST:
                            data = vars_.sst.values
                        if count == 3:
                            # PAR:
                            data = vars_.par.values
                        if count == 4:
                            # Kd (490 nm)
                            data = vars_.Kd_490.values
                        if count == 5:
                            # POC
                            data = vars_.poc.values
                        if count == 6:
                            # FLH
                            data = vars_.nflh.values
                        if count == 7:
                            # iPAR
                            data = vars_.ipar.values
                        if count == 8:
                            # CDOM
                            data = vars_.adg_443_giop.values
                        lat = vars_.coords['lat'].values
                        lon = vars_.coords['lon'].values
                        time = pd.to_datetime(vars_.time_coverage_start)+datetime.timedelta(days=2) # this adds 2 days to correct for SeaWiFS files starting on the last day of previous month
                    if count == 9:
                        # SSHA
                        # Open netCDF file
                        vars_ = xr.open_dataset(filepath)
                        data = vars_.SLA.values[0,:,:].T # transpose to arrange as lat x lon
                        lat = vars_.coords['Latitude'].values
                        lon = vars_.coords['Longitude'].values
                        lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
                        time = pd.to_datetime(vars_.coords['Time'].values)
                    if count == 10:
                        # SSH
                        # Open netCDF file
                        vars_ = xr.open_dataset(filepath)
                        data = vars_.SSH.values[0,:,:] # transpose to arrange as lat x lon
                        lat = vars_.latitude.values
                        lon = vars_.longitude.values
                        time = pd.to_datetime(vars_.time_coverage_start)+datetime.timedelta(days=2) # this adds 2 days to correct for SeaWiFS files starting on the last day of previous month
                    if count == 11:
                        # NPP
                        vars_ = SD(filepath, SDC.READ)
                        data = np.flipud(vars_.select('npp').get())
                        ndeg = 180/1080
                        lat = np.arange(-90,90,ndeg)
                        lon = np.arange(-180,180,ndeg)
                        # extract date from julian day in filename:
                        julian_date = filepath.split(".")[1]
                        time = pd.to_datetime(datetime.datetime.strptime(julian_date[2:], '%y%j').date()) #strftime requires year to be cropped to last 2 digits
                    #-----------------------------------------------------------------
                    # case 1: match pixels to high resolution satellite data
                    if lon[1]-lon[0] >= grid: #i.e. if downsampling to finer grid size
                        # Regrid data and interpolate though:
                        # First find the indices to index by lat/lon
                        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                        
                        # Restrict the data to the NE Pacific lat/lons
                        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                        
                        # Create data matrix with existing coordinates
                        data_mat = pd.DataFrame(data_indexed)
                        data_mat.columns = lon[loninds].flatten()
                        data_mat.index = lat[latinds].flatten()
                        
                        # Create a new matrix with new gridded coordinates
                        lat_new = np.arange(min_lat, max_lat+grid, grid)
                        lon_new = np.arange(min_lon, max_lon+grid, grid)
                        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                        new_shape.columns=lon_new
                        new_shape.index=lat_new
                        
                        # Now reindex exisitng data to new coordinates + add in NaNs to interpolate through later
                        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                        data_proc = data_proc.rename(columns={0:'data'})
                        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                        data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                        data_proc.reset_index(inplace=True)
                        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                        data_proc.reset_index(inplace=True)
                        
                    #-----------------------------------------------------------------
                    # case 2: downsample to courser grid
                    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                        # Bin the data
                        
                        # First find the indices to index by lat/lon
                        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                        
                        # Restrict the data to the NE Pacific lat/lons
                        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                        
                        # Convert time/lats/lons into repeating matrices to match data dimensions
                        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                        time_mat = np.tile(time, len(np.ravel(lat_mat)))
                        
                        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                        # Need these as pandas dataframes to using binning scheme:
                        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                        data_long = pd.DataFrame(data=d)
                        
                        # Bin data as averages across gridded spatial bins:
                        to_bin = lambda x: np.round(x / grid) * grid
                        data_long['latbins'] = data_long.lat.map(to_bin)
                        data_long['lonbins'] = data_long.lon.map(to_bin)
                        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                        
                        # Rename binned columns + drop mean lat/lons:
                        data_proc = data_proc.drop(columns=['lat', 'lon'])
                        data_proc.reset_index(inplace=True) # remove index specification on columns
                    # #-----------------------------------------------------------------
                    # # Bin the data
                    
                    # # First find the indices to index by lat/lon
                    # latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                    # loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                    
                    # # Restrict the data to the NE Pacific lat/lons
                    # data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                    
                    # # Convert time/lats/lons into repeating matrices to match data dimensions
                    # lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                    # lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                    # time_mat = np.tile(time, len(np.ravel(lat_mat)))
                    
                    # # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                    # # Need these as pandas dataframes to use binning scheme:
                    # d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                    # data_long = pd.DataFrame(data=d)
                    
                    # # Bin data as averages across gridded spatial bins:
                    # to_bin = lambda x: np.round(x / grid) * grid
                    # data_long['latbins'] = data_long.lat.map(to_bin)
                    # data_long['lonbins'] = data_long.lon.map(to_bin)
                    # data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                    
                    # # Rename binned columns + drop mean lat/lons:
                    # data_proc = data_proc.drop(columns=['lat', 'lon'])
                    # data_proc.reset_index(inplace=True) # remove index specification on columns
                #-----------------------------------------------------------------
                # Lets extract MIMOC data...
                if count >= 12:
                    if count == 12:
                        # Open netCDF file
                        vars_ = xr.open_dataset(filepath)
                        data = vars_.DEPTH_MIXED_LAYER.values
                    if count == 13:
                        # Open netCDF file
                        vars_ = xr.open_dataset(filepath)
                        data = vars_.SALINITY[0,:,:].values # select values at surface layer (0 m)
                    lat = vars_.LATITUDE.values
                    lon = vars_.LONGITUDE.values
                    lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
                    julian_date = file.split("month")[1].split('.nc')[0]
                    time = pd.to_datetime(datetime.datetime.strptime(julian_date[:2], '%m').date()) #strftime requires year to be cropped to last 2 digits
                    #-----------------------------------------------------------------
                    # case 1: match pixels to high resolution satellite data
                    if lon[1]-lon[0] >= grid: #i.e. if downsampling to finer grid size
                        # Regrid data and interpolate though:
                        # First find the indices to index by lat/lon
                        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                        
                        # Restrict the data to the NE Pacific lat/lons
                        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                        
                        # Create data matrix with existing coordinates
                        data_mat = pd.DataFrame(data_indexed)
                        data_mat.columns = lon[loninds].flatten()
                        data_mat.index = lat[latinds].flatten()
                        
                        # Create a new matrix with new gridded coordinates
                        lat_new = np.arange(min_lat, max_lat+grid, grid)
                        lon_new = np.arange(min_lon, max_lon+grid, grid)
                        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                        new_shape.columns=lon_new
                        new_shape.index=lat_new
                        
                        # Now reindex exisitng data to new coordinates + add in NaNs to interpolate through later
                        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                        data_proc = data_proc.rename(columns={0:'data'})
                        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                        data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                        data_proc.reset_index(inplace=True)
                        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                        data_proc.reset_index(inplace=True)
                        
                    #-----------------------------------------------------------------
                    # case 2: downsample to courser grid
                    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                        # Bin the data
                        
                        # First find the indices to index by lat/lon
                        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                        
                        # Restrict the data to the NE Pacific lat/lons
                        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                        
                        # Convert time/lats/lons into repeating matrices to match data dimensions
                        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                        time_mat = np.tile(time, len(np.ravel(lat_mat)))
                        
                        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                        # Need these as pandas dataframes to using binning scheme:
                        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                        data_long = pd.DataFrame(data=d)
                        
                        # Bin data as averages across gridded spatial bins:
                        to_bin = lambda x: np.round(x / grid) * grid
                        data_long['latbins'] = data_long.lat.map(to_bin)
                        data_long['lonbins'] = data_long.lon.map(to_bin)
                        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                        
                        # Rename binned columns + drop mean lat/lons:
                        data_proc = data_proc.drop(columns=['lat', 'lon'])
                        data_proc.reset_index(inplace=True) # remove index specification on columns
                            
                # Save to output array
                if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
                    variables[k] = dd.from_pandas(data_proc, chunksize=10)
                else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
                    variables[k] = dd.concat([variables[k],dd.from_pandas(data_proc, chunksize=10)])
                 
                # Print status: completed iteration
                print(folder + ' (' + str(count+1) + ' of ' + str(np.size(folders)) + ' folders)' + ': Completed ' + str(n+1) + ' of ' + str(np.size(files)))
        print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('Ta-da! ' + str(count+1) + ' of ' + str(np.size(folders)) + ' datasets processed')
end = timeit.default_timer() # stop the clock
print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~')
print('Execution time:') # output computation times:
print(str(round(end-start,5)),'secs')
print(str(round((end-start)/60,5)),'mins')
print(str(round((end-start)/3600,5)),'hrs')
del vars_, data_long, data, data_indexed, data_proc, time_mat

## Extracting Copernicus Wind Data

In [ ]:

print('\nExtracting Copernicus wind field data...')
print('\nLoading data...')
# Pre-allocate dask array to store new binned data to
wind = dd.from_array(da.empty((1,6)))
# Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
windfile = xr.open_dataset('E:/Satellite/CERSAT-GLO-REP_WIND_L4-OBS_FULL_TIME_SERIE_1607246388558.nc')

# Define variables
# shape is (time,depth,lat,lon)
lat = windfile.coords['latitude'].values
lon = windfile.coords['longitude'].values
U = windfile.eastward_wind.values[:,0,:,:]
V = windfile.northward_wind.values[:,0,:,:]
wspd = windfile.wind_speed.values[:,0,:,:]
timerange = pd.to_datetime(windfile.time.values,utc=True).strftime('%Y%m').to_series().reset_index(drop=True)


# Define indices
latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
PMEL = pd.read_csv('C:/Users/bcamc/OneDrive/Desktop/Python/Projects/py_eosc510/Final Project/NEPac_Data/PMEL_Herr_all_data.csv')
timeinds = np.unique(pd.to_datetime(PMEL['DateTime']).dt.strftime('%Y%m'))[1:]

idx = np.arange(0,timerange.size,1)
times = pd.DataFrame(idx,index=timerange)
# 1 indexes from '200706' onwards
timepts = times.loc[timeinds[1:]].values


print('\nBegin extraction loop...')
# Loop to bin/reshape
for n,pt in enumerate(timepts[:,0]):
    print(str(n+1)+' of '+str(np.size(timepts[:,0]))+' iterations...')
    # index data
    wspd_indexed = wspd[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
    U_indexed = U[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
    V_indexed = V[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
    
    # Convert time/lats/lons into repeating matrices to match data dimensions
    lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
    lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
    time_mat = np.tile(timerange[pt], len(np.ravel(lat_mat)))
    
    # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
    # Need these as pandas dataframes to using binning scheme:
    d = {'datetime':time_mat,
         'lat':np.ravel(lat_mat),
         'lon':np.ravel(lon_mat),
         'wspd':np.ravel(wspd_indexed),
         'U':np.ravel(U_indexed),
         'V':np.ravel(V_indexed)}
    data_long = pd.DataFrame(data=d)
    
    # Since data is already at 0.25x0.25 scale, "bin" by shifting coords by 0.125 to match other variable coordinates
    data_long.loc[:,'lat'] = data_long.loc[:,'lat']+0.125
    data_long.loc[:,'lon'] = data_long.loc[:,'lon']+0.125
                
    # Save to output array
    if n == 0:
        wind = dd.from_pandas(data_long, chunksize=10)
    else:
        wind = dd.concat([wind,dd.from_pandas(data_long, chunksize=10)])
    print(str(n+1)+' of '+str(np.size(timepts[:,0]))+' iterations completed!')
del windfile

## Processing WOA18 data

In [ ]:

print('\nProcessing WOA18 SSN data...')
print('\nLoading data...')
#------------------------------------------------------------------------------
#### Nitrate
#------------------------------------------------------------------------------
# Pre-allocate dask array to store new binned data to
SSN = dd.from_array(da.empty((1,3)))

for n,pt in enumerate(np.array([6,7,8])):
    # Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
    data = pd.read_csv('E:/woa18_all_n0'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    lon = data.index.get_level_values('lon')
    lat = data.index.get_level_values('lat')
    data = data.unstack()
    time = pd.to_datetime({'year': [2000],'month': [pt],'day': [1]})
    
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if downsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        # latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        # loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]
        
        # Create data matrix with existing coordinates
        data_mat = pd.DataFrame(data_indexed)
        # data_mat.columns = lon[min_lon:max_lon]
        # data_mat.index = lat[min_lat:max_lat]
        
        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new
        
        # Now reindex exisitng data to new coordinates + add in NaNs to interpolate through later
        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(time,data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        data_proc.reset_index(inplace=True)

    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(time, len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
        data_proc.reset_index(inplace=True) # remove index specification on columns

    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        SSN = dd.from_pandas(data_proc, chunksize=10)
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        SSN = dd.concat([SSN,dd.from_pandas(data_proc, chunksize=10)])
    print(str(n+1)+' of '+str(np.size(np.array([6,7,8])))+' iterations completed!')

#------------------------------------------------------------------------------
#### Phosphate
#------------------------------------------------------------------------------
# Pre-allocate dask array to store new binned data to
SSP = dd.from_array(da.empty((1,3)))

for n,pt in enumerate(np.array([6,7,8])):
    # Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
    data = pd.read_csv('E:/woa18_all_p0'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    lon = data.index.get_level_values('lon')
    lat = data.index.get_level_values('lat')
    data = data.unstack()
    time = pd.to_datetime({'year': [2000],'month': [pt],'day': [1]})
    
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if downsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        # latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        # loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]
        
        # Create data matrix with existing coordinates
        data_mat = pd.DataFrame(data_indexed)
        # data_mat.columns = lon[min_lon:max_lon]
        # data_mat.index = lat[min_lat:max_lat]
        
        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new
        
        # Now reindex exisitng data to new coordinates + add in NaNs to interpolate through later
        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(time,data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        data_proc.reset_index(inplace=True)

    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(time, len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
        data_proc.reset_index(inplace=True) # remove index specification on columns

    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        SSP = dd.from_pandas(data_proc, chunksize=10)
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        SSP = dd.concat([SSP,dd.from_pandas(data_proc, chunksize=10)])
    print(str(n+1)+' of '+str(np.size(np.array([6,7,8])))+' iterations completed!')

## ETOPO2 Bathymetry Data

In [ ]:
# This bathymetry data is used to mask out land pixels when interpolating
vars_ = xr.open_dataset('E:/etopo2.nc')
data = vars_.btdata.values
lat = vars_.coords['lat'].values
lon = vars_.coords['lon'].values
#-----------------------------------------------------------------
# Bin the data

# First find the indices to index by lat/lon
latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

# Restrict the data to the NE Pacific lat/lons
data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

# Convert time/lats/lons into repeating matrices to match data dimensions
lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
time_mat = np.tile(time, len(np.ravel(lat_mat)))

# Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
# Need these as pandas dataframes to using binning scheme:
d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
data_long = pd.DataFrame(data=d)

# Bin data as averages across gridded spatial bins:
to_bin = lambda x: np.round(x / grid) * grid
data_long['latbins'] = data_long.lat.map(to_bin)
data_long['lonbins'] = data_long.lon.map(to_bin)
data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()

# Rename binned columns + drop mean lat/lons:
data_proc = data_proc.drop(columns=['lat', 'lon'])
data_proc.reset_index(inplace=True) # remove index specification on columns#-----------------------------------------------------------------
# Bin the data

# First find the indices to index by lat/lon
latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

# Restrict the data to the NE Pacific lat/lons
data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

# Convert time/lats/lons into repeating matrices to match data dimensions
lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
time_mat = np.tile(time, len(np.ravel(lat_mat)))

# Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
# Need these as pandas dataframes to using binning scheme:
d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
data_long = pd.DataFrame(data=d)

# Bin data as averages across gridded spatial bins:
to_bin = lambda x: np.round(x / grid) * grid
data_long['latbins'] = data_long.lat.map(to_bin)
data_long['lonbins'] = data_long.lon.map(to_bin)
data_proc = data_long.groupby(['datetime', 'lonbins', 'latbins']).mean()

# Rename binned columns + drop mean lat/lons:
data_proc = data_proc.drop(columns=['lat', 'lon'])
data_proc.reset_index(inplace=True) # remove index specification on columns
etopo = data_proc.iloc[:,1:]
etopo.set_index(['lonbins','latbins'], inplace=True)
del data_proc, vars_, data_long

## Reshape Data

In [ ]:

#------------------------------------------------------------------------------
# NOTE: If extracting one variable at a time above, then code below needs to run
# line-by-line for that specific variable (for both reshaping + interpolating)
#------------------------------------------------------------------------------

#### Reshape data
# Pivot to move lat/lon pairs to columns
# reset_index pulls the dates back into a column
print('Reshaping satellite data...')
CHL_sat = variables['chl'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
CHL_sat = CHL_sat.groupby(CHL_sat['datetime'].dt.strftime('%m')).mean()
# del variables['chl']
print('CHL done')
SSH_sat = variables['SSH'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SSH_sat = SSH_sat.groupby(SSH_sat['datetime'].dt.strftime('%m')).mean()
# del variables['SSHA']
print('SSH done')
SSHA_sat = variables['SSHA'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SSHA_sat = SSHA_sat.groupby(SSHA_sat['datetime'].dt.strftime('%m')).mean()
# del variables['SSHA']
print('SSHA done')
SST_sat = variables['SST'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SST_sat = SST_sat.groupby(SST_sat['datetime'].dt.strftime('%m')).mean()
# del variables['SST']
print('SST done')
PIC_sat = variables['PIC'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
PIC_sat = PIC_sat.groupby(PIC_sat['datetime'].dt.strftime('%m')).mean()
# del variables['PIC']
print('PIC done')
PAR_sat = variables['PAR'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
PAR_sat = PAR_sat.groupby(PAR_sat['datetime'].dt.strftime('%m')).mean()
# del variables['PAR']
print('PAR done')
Kd_sat = variables['Kd'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
Kd_sat = Kd_sat.groupby(Kd_sat['datetime'].dt.strftime('%m')).mean()
# del variables['Kd']
print('Kd done')
POC_sat = variables['POC'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
POC_sat = POC_sat.groupby(POC_sat['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('POC done')
FLH_sat = variables['FLH'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
FLH_sat = FLH_sat.groupby(FLH_sat['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('FLH done')
iPAR_sat = variables['iPAR'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
iPAR_sat = iPAR_sat.groupby(iPAR_sat['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('iPAR done')
CDOM_sat = variables['CDOM'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
CDOM_sat = CDOM_sat.groupby(CDOM_sat['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('CDOM done')
SSN_woa = SSN.compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SSN_woa = SSN_woa.groupby(SSN_woa['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('SSN done')
SSP_woa = SSP.compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SSP_woa = SSP_woa.groupby(SSP_woa['datetime'].dt.strftime('%m')).mean()
# del variables['POC']
print('SSP done')
NPP_sat = variables['NPP'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
NPP_sat = NPP_sat.groupby(NPP_sat['datetime'].dt.strftime('%m')).mean()
NPP_sat = NPP_sat.replace(-9999,'NaN').astype(float)
NPP_sat = NPP_sat.reindex_like(CHL_sat)
# del variables['NPP']
print('NPP done')
MLD = variables['MLD'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
MLD['datetime'] = MLD['datetime'].dt.strftime('%m')
MLD.set_index('datetime', inplace=True)
# del variables['MLD']
print('MLD done')
SAL_argo = variables['SAL'].compute().pivot(index='datetime',columns=['latbins','lonbins'], values='data').reset_index()
SAL_argo['datetime'] = SAL_argo['datetime'].dt.strftime('%m')
SAL_argo.set_index('datetime', inplace=True)
# del variables['SAL']
print('SAL done')
WSPD_sat = wind.compute().pivot(index='datetime',columns=['lat','lon'], values='wspd').reset_index()
WSPD_sat['datetime'] = pd.to_datetime(WSPD_sat['datetime'], format='%Y%m')
WSPD_sat = WSPD_sat.groupby(WSPD_sat['datetime'].dt.strftime('%m')).mean()
WSPD_sat = WSPD_sat.reindex_like(CHL_sat)
print('WSPD done')
U_sat = wind.compute().pivot(index='datetime',columns=['lat','lon'], values='U').reset_index()
U_sat['datetime'] = pd.to_datetime(U_sat['datetime'], format='%Y%m')
U_sat = U_sat.groupby(U_sat['datetime'].dt.strftime('%m')).mean()
U_sat = U_sat.reindex_like(CHL_sat)
print('U done')
V_sat = wind.compute().pivot(index='datetime',columns=['lat','lon'], values='V').reset_index()
V_sat['datetime'] = pd.to_datetime(V_sat['datetime'], format='%Y%m')
V_sat = V_sat.groupby(V_sat['datetime'].dt.strftime('%m')).mean()
V_sat = V_sat.reindex_like(CHL_sat)
print('V done')
#------------------------------------------------------------------------------

## Interpolate Data

In [ ]:
#### Interpolate through the NaNs
print()
print('Interpolating data...')
func='linear'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### Chl:
    
# stack to column vectors:
CHL_stack = CHL_sat.stack(dropna=False).stack(dropna=False)
CHL_stack_dropna = CHL_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = CHL_stack_dropna.index
ind_to_interp = CHL_stack.index
d = CHL_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in CHL_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in CHL_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
CHL_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'CHL':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
CHL_sat_interp = CHL_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='CHL').reindex_like(CHL_sat)

# Restrict chlorophyll to limit of detection:
# CHL_sat_interp[CHL_sat_interp<0.01]=0.01
print('Chlorophyll done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### PIC:
    
# stack to column vectors:
PIC_stack = PIC_sat.stack(dropna=False).stack(dropna=False)
PIC_stack_dropna = PIC_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = PIC_stack_dropna.index
ind_to_interp = PIC_stack.index
d = PIC_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in PIC_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in PIC_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
PIC_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'PIC':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
PIC_sat_interp = PIC_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='PIC').reindex_like(CHL_sat)

print('PIC done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### PAR:
    
# stack to column vectors:
PAR_stack = PAR_sat.stack(dropna=False).stack(dropna=False)
PAR_stack_dropna = PAR_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = PAR_stack_dropna.index
ind_to_interp = PAR_stack.index
d = PAR_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in PAR_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in PAR_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
PAR_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'PAR':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
PAR_sat_interp = PAR_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='PAR').reindex_like(CHL_sat)
# PAR_sat_interp[PAR_sat_interp<0]=0
print('PAR done')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### SSH:
    
# stack to column vectors:
SSH_stack = SSH_sat.stack(dropna=False).stack(dropna=False)
SSH_stack_dropna = SSH_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SSH_stack_dropna.index
ind_to_interp = SSH_stack.index
d = SSH_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SSH_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SSH_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SSH_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SSH':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
SSH_sat_interp = SSH_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SSH').reindex_like(CHL_sat)

print('SSH done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### SSHA:
    
# stack to column vectors:
SSHA_stack = SSHA_sat.stack(dropna=False).stack(dropna=False)
SSHA_stack_dropna = SSHA_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SSHA_stack_dropna.index
ind_to_interp = SSHA_stack.index
d = SSHA_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SSHA_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SSHA_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SSHA_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SSHA':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
SSHA_sat_interp = SSHA_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SSHA').reindex_like(CHL_sat)

print('SSHA done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### SST:
    
# stack to column vectors:
SST_stack = SST_sat.stack(dropna=False).stack(dropna=False)
SST_stack_dropna = SST_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SST_stack_dropna.index
ind_to_interp = SST_stack.index
d = SST_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SST_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SST_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SST_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SST':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
SST_sat_interp = SST_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SST').reindex_like(CHL_sat)
print('SST done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### Kd:
    
# stack to column vectors:
Kd_stack = Kd_sat.stack(dropna=False).stack(dropna=False)
Kd_stack_dropna = Kd_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = Kd_stack_dropna.index
ind_to_interp = Kd_stack.index
d = Kd_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in Kd_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in Kd_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
Kd_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'Kd':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
Kd_sat_interp = Kd_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='Kd').reindex_like(CHL_sat)

print('Kd done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### POC:
    
# stack to column vectors:
POC_stack = POC_sat.stack(dropna=False).stack(dropna=False)
POC_stack_dropna = POC_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = POC_stack_dropna.index
ind_to_interp = POC_stack.index
d = POC_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in POC_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in POC_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
POC_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'POC':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
POC_sat_interp = POC_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='POC').reindex_like(CHL_sat)

print('POC done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### FLH:
    
# stack to column vectors:
FLH_stack = FLH_sat.stack(dropna=False).stack(dropna=False)
FLH_stack_dropna = FLH_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = FLH_stack_dropna.index
ind_to_interp = FLH_stack.index
d = FLH_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in FLH_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in FLH_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
FLH_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'FLH':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
FLH_sat_interp = FLH_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='FLH').reindex_like(CHL_sat)

print('FLH done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### iPAR:
    
# stack to column vectors:
iPAR_stack = iPAR_sat.stack(dropna=False).stack(dropna=False)
iPAR_stack_dropna = iPAR_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = iPAR_stack_dropna.index
ind_to_interp = iPAR_stack.index
d = iPAR_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in iPAR_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in iPAR_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
iPAR_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'iPAR':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
iPAR_sat_interp = iPAR_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='iPAR').reindex_like(CHL_sat)

print('iPAR done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### CDOM:
    
# stack to column vectors:
CDOM_stack = CDOM_sat.stack(dropna=False).stack(dropna=False)
CDOM_stack_dropna = CDOM_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = CDOM_stack_dropna.index
ind_to_interp = CDOM_stack.index
d = CDOM_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in CDOM_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in CDOM_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
CDOM_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'CDOM':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
CDOM_sat_interp = CDOM_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='CDOM').reindex_like(CHL_sat)

print('CDOM done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### SSN:
    
# stack to column vectors:
SSN_stack = SSN_woa.stack(dropna=False).stack(dropna=False)
SSN_stack_dropna = SSN_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SSN_stack_dropna.index
ind_to_interp = SSN_stack.index
d = SSN_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SSN_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SSN_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SSN_woa_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SSN':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
SSN_woa_interp = SSN_woa_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SSN').reindex_like(CHL_sat)

print('SSN done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### SSP:
    
# stack to column vectors:
SSP_stack = SSP_woa.stack(dropna=False).stack(dropna=False)
SSP_stack_dropna = SSP_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SSP_stack_dropna.index
ind_to_interp = SSP_stack.index
d = SSP_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SSP_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SSP_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SSP_woa_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SSP':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
SSP_woa_interp = SSP_woa_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SSP').reindex_like(CHL_sat)

print('SSP done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### NPP:
    
# stack to column vectors:
NPP_stack = NPP_sat.stack(dropna=False).stack(dropna=False)
NPP_stack_dropna = NPP_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = NPP_stack_dropna.index
ind_to_interp = NPP_stack.index
d = NPP_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in NPP_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in NPP_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
NPP_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'NPP':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
NPP_sat_interp = NPP_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='NPP').reindex_like(CHL_sat)
print('NPP done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### MLD:
    
# stack to column vectors:
MLD_stack = MLD.stack(dropna=False).stack(dropna=False)
MLD_stack_dropna = MLD_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = MLD_stack_dropna.index
ind_to_interp = MLD_stack.index
d = MLD_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in MLD_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in MLD_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
MLD_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'MLD':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
MLD_interp = MLD_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='MLD').reindex_like(CHL_sat)

print('MLD done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### Salinity:
    
# stack to column vectors:
SAL_argo_stack = SAL_argo.stack(dropna=False).stack(dropna=False)
SAL_argo_stack_dropna = SAL_argo_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = SAL_argo_stack_dropna.index
ind_to_interp = SAL_argo_stack.index
d = SAL_argo_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in SAL_argo_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in SAL_argo_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
SAL_argo_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'SAL':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
SAL_argo_interp = SAL_argo_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='SAL').reindex_like(CHL_sat)

print('SAL done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### WSPD:
    
# stack to column vectors:
WSPD_stack = WSPD_sat.stack(dropna=False).stack(dropna=False)
WSPD_stack_dropna = WSPD_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = WSPD_stack_dropna.index
ind_to_interp = WSPD_stack.index
d = WSPD_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in WSPD_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in WSPD_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
WSPD_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'WSPD':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})

# Reshape and filter out negative interpolated values
WSPD_sat_interp = WSPD_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='WSPD').reindex_like(CHL_sat)
WSPD_sat_interp[WSPD_sat_interp<0]=0
print('WSPD done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### U:
    
# stack to column vectors:
U_stack = U_sat.stack(dropna=False).stack(dropna=False)
U_stack_dropna = U_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = U_stack_dropna.index
ind_to_interp = U_stack.index
d = U_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in U_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in U_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
U_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'U':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
U_sat_interp = U_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='U').reindex_like(CHL_sat)
print('U done')
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#### V:
    
# stack to column vectors:
V_stack = V_sat.stack(dropna=False).stack(dropna=False)
V_stack_dropna = V_stack.dropna()

# Get index and values to build interpolation function
ind_from_raw = V_stack_dropna.index
ind_to_interp = V_stack.index
d = V_stack_dropna.values
d = da.from_array(d, chunks=(d.shape[0]))

date_ind = np.asarray([x[0] for x in V_stack.index]).astype(float)
date_ind_raw = np.asarray([x[0] for x in V_stack_dropna.index]).astype(float)

# Build RBF interpolation functions:
rbfinterp6 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 6],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 6],
                                  d[date_ind_raw == 6],function='linear')

rbfinterp7 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 7],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 7],
                                  d[date_ind_raw == 7],function='linear')

rbfinterp8 = scipy.interpolate.Rbf(da.array([n[0] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[1] for n in ind_from_raw])[date_ind_raw == 8],
                                  da.array([n[2] for n in ind_from_raw])[date_ind_raw == 8],
                                  d[date_ind_raw == 8],function='linear')

# Interpolate values:
V_sat_interp = pd.DataFrame({'DateTime':np.array([n[0] for n in ind_to_interp]),
                            'Lon':np.array([n[1] for n in ind_to_interp]),
                            'Lat':np.array([n[2] for n in ind_to_interp]),
                            'V':np.concatenate([rbfinterp6(np.array([n[0] for n in ind_to_interp])[date_ind == 6],np.array([n[1] for n in ind_to_interp])[date_ind == 6],np.array([n[2] for n in ind_to_interp])[date_ind == 6]),
                                             rbfinterp7(np.array([n[0] for n in ind_to_interp])[date_ind == 7],np.array([n[1] for n in ind_to_interp])[date_ind == 7],np.array([n[2] for n in ind_to_interp])[date_ind == 7]),
                                             rbfinterp8(np.array([n[0] for n in ind_to_interp])[date_ind == 8],np.array([n[1] for n in ind_to_interp])[date_ind == 8],np.array([n[2] for n in ind_to_interp])[date_ind == 8])],axis=0)})


# Reshape and filter out negative interpolated values
V_sat_interp = V_sat_interp.pivot(index='DateTime',
                                      columns=['Lat','Lon'],
                                      values='V').reindex_like(CHL_sat)
print('V done')
#-----------------------------------------------------------------------------
# Clear some space in the memory
del ind_from_raw, ind_to_interp, d, date_ind, date_ind_raw, rbfinterp6, rbfinterp7, rbfinterp8
#-----------------------------------------------------------------------------

## Write processed data to files

In [ ]:
#-----------------------------------------------------------------------------
# Write files
CHL_sat.to_csv(write_dir+'CHL_sat_'+str(grid)+'.csv') # for reshaping

etopo.to_csv(write_dir+'etopo2_'+str(grid)+'.csv')

CHL_sat_interp.to_csv(write_dir+'CHL_sat_interp_'+str(grid)+'.csv')
PAR_sat_interp.to_csv(write_dir+'PAR_sat_interp_'+str(grid)+'.csv')
PIC_sat_interp.to_csv(write_dir+'PIC_sat_interp_'+str(grid)+'.csv')
SSHA_sat_interp.to_csv(write_dir+'SSHA_sat_interp_'+str(grid)+'.csv')
SSH_sat_interp.to_csv(write_dir+'SSH_sat_interp_'+str(grid)+'.csv')
SST_sat_interp.to_csv(write_dir+'SST_sat_interp_'+str(grid)+'.csv')
Kd_sat_interp.to_csv(write_dir+'Kd_sat_interp_'+str(grid)+'.csv')
POC_sat_interp.to_csv(write_dir+'POC_sat_interp_'+str(grid)+'.csv')
FLH_sat_interp.to_csv(write_dir+'FLH_sat_interp_'+str(grid)+'.csv')
iPAR_sat_interp.to_csv(write_dir+'iPAR_sat_interp_'+str(grid)+'.csv')
CDOM_sat_interp.to_csv(write_dir+'CDOM_sat_interp_'+str(grid)+'.csv')
SSN_woa_interp.to_csv(write_dir+'SSN_woa_interp_'+str(grid)+'.csv')
SSP_woa_interp.to_csv(write_dir+'SSP_woa_interp_'+str(grid)+'.csv')
NPP_sat_interp.to_csv(write_dir+'NPP_sat_interp_'+str(grid)+'.csv')
MLD_interp.to_csv(write_dir+'MLD_argo_interp_'+str(grid)+'.csv')
SAL_argo_interp.to_csv(write_dir+'SAL_argo_interp_'+str(grid)+'.csv')
WSPD_sat_interp.to_csv(write_dir+'WSPD_sat_interp_'+str(grid)+'.csv')
U_sat_interp.to_csv(write_dir+'U_sat_interp_'+str(grid)+'.csv')
V_sat_interp.to_csv(write_dir+'V_sat_interp_'+str(grid)+'.csv')
##-----------------------------------------------------------------------------

## Script runtime

In [ ]:
runtime_end = timeit.default_timer() # stop the clock
extraction_runtime = runtime_end-runtime_start
print('\nData Extraction Runtime:')
print(str(round(extraction_runtime,5)),'secs')
print(str(round((extraction_runtime)/60,5)),'mins')
print(str(round((extraction_runtime)/3600,5)),'hrs')